<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.3:
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands".

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [ ]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [ ]:
pwd

'/content'

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [ ]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

OperationalError: ignored

In [ ]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitutionusing the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

In [ ]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

In [ ]:
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
#     this is a security measure against SQL injection attacks:
c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

In [ ]:
c.execute('select * from houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more.

In [ ]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('../DATA/housing-data.csv', dtype = 'i8',
                    delimiter = ',', skip_header = 1).tolist()

FileNotFoundError: ignored

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [ ]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [ ]:
# check:
print(type(data))
print(data[0:3])

Now we can insert each list item as a row of fields in the database.

In [ ]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [ ]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

OperationalError: ignored

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [ ]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

ANSWER: Delete all records from the table `houses`.

#### Filtering Rows

In [ ]:
# similar syntax as before
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# here results is a cursor object - use fetchall() to extract a list
results.fetchall()

OperationalError: ignored

Run a query to show the effect of the DELETE command that was executed above (HINT: You can use the WHERE clause to reduce the size of the rowset to the region of interest):

In [ ]:
# ANSWER
results = c.execute("SELECT * FROM houses WHERE field1 >= 50 AND field1 < 59")
results.fetchall()

Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):

In [ ]:
#ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [ ]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database.

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [ ]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('../DATA/housing-data.csv', low_memory = False)
data.head()

We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [ ]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [ ]:
#ANSWER
c.execute('SELECT bdrms, AVG(price) FROM houses_pandas GROUP BY bdrms').fetchall()

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

**Answer**
1. Pandas is convenient for cleaning data incrementally, where it is easy to see intermediate steps. Once clean the dataset can be stored in a database.
2. Pandas might be considered more amenable to statistical analysis, and can be used to prepare data for plotting.

<a name="guided-practice"></a>


# Reference: SQL Syntax


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [ ]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [ ]:
# Reading CSV to Dataframe
orders = pd.read_csv('../DATA/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('../DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [ ]:
orders.head()

In [ ]:
orders_break_down.head()

#### 2. Replace Space with Underscore in Column Names

In [ ]:
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [ ]:
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes

In [ ]:
orders.dtypes

In [ ]:
orders_break_down.dtypes

#### 4. Save these two dataframes as a table in sqlite

In [ ]:
# Establishing Local DB connection
db_connection = sqlite3.connect('../DATA/eshop.db.sqlite')

## Reading out DataFrames as SQL Table
orders.to_sql(name = 'orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql(name = 'orders_break_down', con = db_connection, if_exists = 'replace', index = False)

#### 5. Select Number of Orders for Each Customer

In [ ]:
# Select Number of DISTINCT Orders for Each Customer

sql_string = '''
    SELECT
        customer_name,
        COUNT(DISTINCT orders.order_id) AS Count
    FROM orders
    JOIN orders_break_down ON orders_break_down.order_id = orders.order_id
    GROUP BY customer_name ORDER BY Count DESC
'''

sql_string = '''
    SELECT
        customer_name,
        COUNT(DISTINCT order_id) AS Count
    FROM orders
    GROUP BY customer_name
'''

customers = sql.read_sql(sql_string, con = db_connection)

customers.head()

#### 6. Select Number of Customers for Each Country

In [ ]:
# Select Number of Customers for Each Country

query = '''
    SELECT
        country,
        COUNT(DISTINCT customer_name) AS Count
    FROM orders
    GROUP BY country
    ORDER BY Count DESC
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [ ]:
# Select discount, sales, quantity for Each Order from orders_break_down Table

sql_string = '''
    SELECT
        order_id
        , discount
        , sales
        , quantity
    FROM orders_break_down
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [ ]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table

sql_string = '''
    SELECT
        order_id
        , discount
        , sales
        , quantity
        , (sales * quantity - discount) AS total_price
    FROM orders_break_down
    ORDER BY total_price DESC
'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [ ]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

sql_string = '''
    SELECT
        *
        , (sales * quantity - discount) AS total_price
    FROM orders_break_down
    WHERE sales * quantity - discount > 100
    ORDER BY total_price ASC

'''

customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

#### 8. Select All Customer And The Product They Have Bought

In [ ]:
# Select All Customer And The Product They Have Bought

sql_string = '''
    SELECT
        orders.order_id
        , orders.customer_name
        , orders_break_down.product_name
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
'''
customers = sql.read_sql(sql_string, con = db_connection)
customers.head()

#### 9.A Select Number of 'Furniture' Orders For Each Country

In [ ]:
# Select Number of 'Furniture' Orders For Each Country

sql_string = '''
    SELECT
        orders.country
        , COUNT(orders_break_down.category) AS Count
    FROM orders
    JOIN orders_break_down
    ON orders.order_id = orders_break_down.order_id
    WHERE orders_break_down.category = 'Furniture'
    GROUP BY orders.country
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [ ]:
# Select Number of 'Furniture' Orders For The Country Denmark

sql_string = '''
    SELECT
        *
    FROM orders
    JOIN orders_break_down
    ON orders.order_id = orders_break_down.order_id
    WHERE orders_break_down.category = 'Furniture'
        AND orders.country = 'Denmark'
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [ ]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country

sql_string = '''
    SELECT
        orders.country
        , SUM(CASE WHEN discount == 0 THEN sales ELSE 0 END) AS non_discount_sales
        , SUM(CASE WHEN discount > 0 THEN sales ELSE 0 END) AS discount_sales
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

#### 11.A Select Total Quantity, Total Sales for Each Country

In [ ]:
# Select Total Quantity, Total Sales for Each Country

sql_string = '''
    SELECT
        orders.country
        , SUM(sales) AS total_sales
        , SUM(quantity) AS total_quantity
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

#### 11.B Select Top 3 Countries and Quantities Based on Total Sales

In [ ]:
# Select Top 3 Countries and Quantities Based on Total Sales

sql_string = '''
    SELECT
        orders.country
        , SUM(sales) AS total_sales
        , SUM(quantity) AS total_quantity
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
    ORDER BY total_sales DESC LIMIT 3
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

In [ ]:
order_list.sort_values(by='total_sales', ascending=False).head(3)

#### 11.C Select Bottom 3 Country Based On Quantities

In [ ]:
# Select Bottom 3 Countries and Sales Based On Total Quantities

sql_string = '''
    SELECT
        orders.country
        , SUM(sales) AS total_sales
        , SUM(quantity) AS total_quantity
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
    ORDER BY total_quantity ASC LIMIT 3
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

In [ ]:
order_list.sort_values(by='total_quantity', ascending=True).head(3)

#### 12. Select Average Sales By Category For The Country 'France'

In [ ]:
# Select Average Sales By Category For The Country 'France'

sql_string = '''
    SELECT
        orders.country
        , orders_break_down.category
        , AVG(sales) AS avg_sales
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    WHERE orders.country = 'France'
    GROUP BY orders.country, orders_break_down.category
    ORDER BY avg_sales DESC
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head(100)

#### 13. Select Country, Category and Average Sales Where Average Sales is The Highest

In [ ]:
# Select Country, Category and Average Sales Where Average Sales is The Highest

sql_string = '''
    SELECT
        orders.country
        , orders_break_down.category
        , AVG(sales) AS total_sales
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country, orders_break_down.category
    ORDER BY total_sales DESC LIMIT 1
'''

order_list = sql.read_sql(sql_string, con = db_connection)

order_list.head()

### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams.
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  

>
>
>




---



---



> > > > > > > > > © 2023 Institute of Data


---



---



